In [ ]:
import pandas as pd
import pyodbc
import time

In [ ]:
pip install openpyxl

In [43]:
df = pd.read_excel('data.xlsx')


In [ ]:
df.head()

In [ ]:
# TODO 
# Verileri sql database içerisine aktarıp analiz edelim.
# Pozisyon bazlı yayınlanan iş ilanı 
# Gün bazlı yayınlanan iş ilanı 
# Lokasyon bazlı iş ilanı 

In [ ]:
# Bağlantı bilgileri
server = 'localhost'
database = 'TechcareerDB'
username = 'sa'
password = 'YourStrong!Passw0rd'
driver = '{SQL Server}'  # Sürücü adını güncelledik
schema = 'dbo'  # Şema bilgisi

# Yüklü ODBC sürücülerini kontrol edelim
print("Yüklü ODBC sürücüleri:")
print(pyodbc.drivers())


In [ ]:

# Bağlantı dizesi
conn_str = (
    f'DRIVER={driver};'
    f'SERVER={server};'
    f'DATABASE={database};'
    f'UID={username};'
    f'PWD={password}'
)

# Bağlantıyı test etme
try:
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()
    cursor.execute("SELECT @@VERSION;")
    version = cursor.fetchone()
    print(f"SQL Server bağlantısı başarılı. Sürüm: {version[0]}")
    cursor.close()
    conn.close()
except pyodbc.Error as e:
    print("SQL Server'a bağlanılamadı.")
    print("Hata mesajı:", e)
    exit(1)

In [45]:

# Bağlantıyı tekrar aç
conn = pyodbc.connect(conn_str)
cursor = conn.cursor()

In [46]:

# Veritabanını oluştur (eğer yoksa)
cursor.execute(f"IF DB_ID(N'{database}') IS NULL CREATE DATABASE [{database}]")
cursor.commit()


In [49]:

# Tablo adı ve tam tablo adı
table_name = 'Jobs'
full_table_name = f'[{schema}].[{table_name}]'

# Sütun adları ve veri tiplerini belirleme
columns_with_types = ', '.join([f'[{col}] NVARCHAR(MAX)' for col in df.columns])

# Tabloyu oluştur (eğer yoksa)
create_table_query = f'''
IF OBJECT_ID(N'{schema}.{table_name}', N'U') IS NULL
BEGIN
    CREATE TABLE {full_table_name} (
        {columns_with_types}
    )
END
'''
cursor.execute(create_table_query)
cursor.commit()


In [50]:


# DataFrame'deki NaN değerlerini None ile değiştiriyoruz
df = df.where(pd.notnull(df), None)

# Tarih sütunlarını belirleme
date_columns = df.select_dtypes(include=['datetime', 'datetime64[ns]']).columns.tolist()
print("Tarih sütunları:", date_columns)


# DataFrame'i SQL Server'a yazma (1000'lik parçalar halinde)
columns = ', '.join([f'[{col}]' for col in df.columns])
placeholders = ', '.join(['?' for _ in df.columns])
insert_query = f'INSERT INTO {full_table_name} ({columns}) VALUES ({placeholders})'

data = [tuple(row) for row in df.values]

chunk_size = 5000  # Her seferinde 100 satır ekle
total_rows = len(df)
for start in range(0, 10000, chunk_size): #total_rows
    end = min(start + chunk_size, total_rows)
    chunk_df = df.iloc[start:end]
    chunk_data = [tuple(row) for row in chunk_df.values]
    try:
        cursor.executemany(insert_query, chunk_data)
        conn.commit()
        print(f"{start + 1}-{end} arası satırlar eklendi.")
    except pyodbc.Error as e:
        print(f"{start + 1}-{end} arası satırlar eklenirken hata oluştu.")
        print("Hata mesajı:", e)
        conn.rollback()
        # Problemli satırları yazdırma
        print(chunk_df)
        exit(1)


Tarih sütunları: ['LastApplyDate', 'PublishDate', 'CreateDate']
1-5000 arası satırlar eklenirken hata oluştu.
Hata mesajı: ('22008', '[22008] [Microsoft][ODBC SQL Server Driver]Datetime field overflow (0) (SQLExecDirectW)')
      SiteId     JobNo                        JobTitle  \
0         60    211919                   TEKNIK SERVIS   
1         60    225513     MUHASEBE / FINANS GÖREVLISI   
2         60    289519             ELEKTRIK TEKNISYENI   
3         60    292417                 VINÇ OPERATÖRÜ    
4         60    292421                           SOFÖR   
...      ...       ...                             ...   
4995       4  i4166391  Your connection is not private   
4996       4  i4166397  Your connection is not private   
4997       4  i4166403  Your connection is not private   
4998       4  i4166409  Your connection is not private   
4999       4  i4166415  Your connection is not private   

                            Position       LastApplyDate         Type  \
0     

In [ ]:
conn_str = (
    f"DRIVER={driver};"
    f"SERVER=localhost;"
    f"DATABASE={database};"
    f"UID={username};"
    f"PWD={password}"
)


In [ ]:
import time
import pyodbc

# Maksimum deneme sayısı ve bekleme süresi
max_retries = 10
wait_seconds = 5

for attempt in range(max_retries):
    try:
        conn = pyodbc.connect(conn_str)
        print("SQL Server'a başarılı bir şekilde bağlanıldı.")
        break
    except pyodbc.Error as e:
        print(f"Bağlantı denemesi {attempt + 1}/{max_retries} başarısız oldu. Bekleniyor...")
        time.sleep(wait_seconds)
else:
    print("SQL Server'a bağlanılamadı. Lütfen ayarları kontrol edin.")
    exit(1)
